# 01 — Data Collection (Binance OHLCV)

This notebook fetches OHLCV for top crypto pairs from Binance and stores each field as separate Parquet files under `storage/ohlcv`. All file paths are relative for macOS/Linux.

**Features:**
- Live data from Binance API (when internet is available)
- Mock data mode for offline testing
- Comprehensive error handling
- Data validation and verification

# Collect Full USDT Universe OHLCV Data from BinanceUS

This notebook collects daily OHLCV (Open, High, Low, Close, Volume) data for all USDT trading pairs from BinanceUS using the `python-binance` library. Data is saved as Parquet files for each OHLCV field, which can be used for future model building and analysis.

In [2]:
# STEP-1: Install Required Package
!pip install python-binance --quiet


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


## Fetch and Save OHLCV Data as Parquet

The following cells fetch daily OHLCV data for all USDT trading pairs from BinanceUS, combine them into DataFrames for each field, and save each as a Parquet file. The last cell displays a sample of the close prices.

In [3]:
import pandas as pd
import time
from binance.client import Client as BinanceClient

# -------------------------
# CONFIG
# -------------------------
BINANCE_TLD = 'US'     # use 'US' for Binance.US
API_KEY = ''           # optional
API_SECRET = ''
START_DATE = '2020-01-01'
FREQ = '1d'   # daily candles

# -------------------------
# INIT CLIENT
# -------------------------
client = BinanceClient(api_key=API_KEY, api_secret=API_SECRET, tld=BINANCE_TLD)

# -------------------------
# STEP 1: Get full USDT universe
# -------------------------
info = client.get_exchange_info()
all_symbols = [
    s['symbol'] for s in info['symbols']
    if s['status'] == 'TRADING' and s['quoteAsset'] == 'USDT'
]
print(f"✅ Found {len(all_symbols)} USDT trading pairs on BinanceUS")

✅ Found 188 USDT trading pairs on BinanceUS


In [6]:
# FUNCTION: Fetch OHLCV
def get_binance_ohlcv(symbol, freq, start_str):
    """Fetch OHLCV candles from BinanceUS. Returns DataFrame with Date index."""
    try:
        klines = client.get_historical_klines(symbol, freq, start_str)
        if not klines:
            return None

        df = pd.DataFrame(klines, columns=[
            'open_time','open','high','low','close','volume',
            'close_time','quote_asset_volume','num_trades',
            'taker_base_volume','taker_quote_volume','ignore'
        ])

        df['Date'] = pd.to_datetime(df['open_time'], unit='ms')
        df = df[['Date','open','high','low','close','volume']]
        df[['open','high','low','close','volume']] = df[['open','high','low','close','volume']].astype(float)
        df.set_index('Date', inplace=True)
        return df

    except Exception as e:
        print(f"⚠️ Error fetching {symbol}: {e}")
        return None

---

**Note:**
- Parquet files are saved in the current directory for each OHLCV field (open, high, low, close, volume).
- You can use these files for future model building or analysis.
- Example output below shows the last few rows of the close price DataFrame.

In [7]:
# STEP 2: Loop over ALL symbols
all_data = {}
for symbol in all_symbols:
    df_symbol = get_binance_ohlcv(symbol, FREQ, START_DATE)
    if df_symbol is not None and not df_symbol.empty:
        all_data[symbol] = df_symbol
        print(f"✅ {symbol}: {len(df_symbol)} rows from {df_symbol.index.min().date()} to {df_symbol.index.max().date()}")
    else:
        print(f"⚠️ Skipped {symbol}")
    time.sleep(0.5)  # avoid hitting API limits

✅ BTCUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ ETHUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ ETHUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ XRPUSDT: 1186 rows from 2020-01-01 to 2025-09-27
✅ XRPUSDT: 1186 rows from 2020-01-01 to 2025-09-27
✅ BCHUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ BCHUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ LTCUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ LTCUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ BNBUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ BNBUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ ADAUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ ADAUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ BATUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ BATUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ ETCUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ ETCUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ XLMUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ XLMUSDT: 2097 rows from 2020-01-01 to 2025-09-27
✅ ZRXUSDT: 2097 rows from 2020-

## Example Output

Below is a sample of the last few rows of the close prices DataFrame after fetching and saving the data.

In [10]:
# STEP 3: Combine into panel-like OHLCV DataFrames
OHLC = {}
for field in ['open','high','low','close','volume']:
    OHLC[field] = pd.DataFrame({
        sym: df[field] for sym, df in all_data.items()
    })

# STEP 4: Save Parquet files
for field, df in OHLC.items():
    out_file = f"binance_{field}_daily.parquet"
    df.to_parquet(out_file)
    print(f"Saved {out_file} with shape {df.shape}")

print("\nExample CLOSE prices:")
print(OHLC['close'].tail())

Saved binance_open_daily.parquet with shape (2097, 188)
Saved binance_high_daily.parquet with shape (2097, 188)
Saved binance_low_daily.parquet with shape (2097, 188)
Saved binance_close_daily.parquet with shape (2097, 188)
Saved binance_volume_daily.parquet with shape (2097, 188)

Example CLOSE prices:
              BTCUSDT  ETHUSDT  XRPUSDT  BCHUSDT  LTCUSDT  BNBUSDT  ADAUSDT  \
Date                                                                          
2025-09-23  111917.91  4176.26   2.8288    555.6   106.22  1014.22   0.8062   
2025-09-24  113223.27  4143.00   2.9255    556.1   105.54  1019.27   0.8108   
2025-09-25  108906.81  3856.40   2.7400    537.3   102.03   946.10   0.7630   
2025-09-26  109725.04  4025.19   2.7836    548.1   104.59   960.82   0.7905   
2025-09-27  109670.63  4018.86   2.7836    544.4   103.91   968.83   0.7921   

            BATUSDT  ETCUSDT  XLMUSDT  ...  WLDUSDT  TOSHIUSDT  USELESSUSDT  \
Date                                   ...                    